In [1]:
import pandas as pd 

tmp = pd.read_csv("../data/T6_2_ROOT.DAT", sep=' ')

In [3]:
tmp.head()

,n,s1,s2,p1,p2
0,1,1.11,2.569,3.58,0.760
1,1,1.19,2.928,3.75,0.821
2,1,1.09,2.865,3.93,0.928
3,1,1.25,3.844,3.94,1.009
4,1,1.11,3.027,3.60,0.766


In [9]:
tmp.iloc[2, 1]

1.09

In [ ]:
import numpy as np

root = pd.read_csv("../data/T6_2_ROOT.DAT", sep=' ', names=['Tree.Number', 'Trunk.Girth.4.Years', 'Ext.Growth.4.Years', 'Trunk.Girth.15.Years', 'Weight.Above.Ground.15.Years'])

In [ ]:
R=root.iloc[:, 1:5].corr().values

In [ ]:
R

In [ ]:
n = R.shape[0]

In [ ]:
Rn = np.linalg.solve(R, np.identity(n))

In [ ]:
smc = 1 - 1/np.diag(Rn)

In [ ]:
smc

In [ ]:
di = np.diag_indices(n)

In [ ]:
R[di] = smc

In [ ]:
R

In [ ]:
w, v = np.linalg.eig(R)

In [ ]:
w

In [ ]:
v

In [ ]:
totprop = 0
for i in w:
    totprop += i / np.sum(w)
    print(totprop)

In [ ]:
lam = v[:,:2]@np.diag(np.sqrt(w[:2]))

In [ ]:
lam**2

In [ ]:
h2 = np.sum(lam**2, axis=1)

In [ ]:
h2

In [ ]:
u2 = 1 -  h2

In [ ]:
com = np.sum(lam**2, axis=1)**2 / np.sum(lam**4, axis=1)

In [ ]:
com

In [ ]:
corpa = pd.DataFrame(np.column_stack((np.round(lam[:,0], 2),
                      np.round(lam[:,1], 2),
                      np.round(h2, 2),
                      np.round(u2, 3),
                      np.round(com, 1))),
                      columns=['PA1', 'PA2', 'h2', 'u2', 'com'])

In [ ]:
corpa

varimax matrix rotation from wikipedia https://en.wikipedia.org/wiki/Talk:Varimax_rotation

In [ ]:
def varimax(Phi, gamma = 1.0, q = 20, tol = 1e-6):
    from scipy import eye, asarray, dot, sum
    p,k = Phi.shape
    R = eye(k)
    d=0
    for i in range(q):
        d_old = d
        Lambda = dot(Phi, R)
        u,s,vh = np.linalg.svd(dot(Phi.T,asarray(Lambda)**3 - (gamma/p) * dot(Lambda, np.diag(np.diag(dot(Lambda.T,Lambda))))))
        R = dot(u,vh)
        d = sum(s)
        if d_old!=0 and d/d_old < 1 + tol: break
    return dot(Phi, R)


In [ ]:
varimax(lam[:,0:2])